<a href="https://colab.research.google.com/github/SanjayBista1010/DeepLearning/blob/main/PytorchSnake%26SpiderColor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import zipfile
import os

# Path to your ZIP file in Google Drive
zip_path = '/content/drive/MyDrive/dataset.zip'

# Destination folder in Colab
extract_path = '/content/images'
os.makedirs(extract_path, exist_ok=True)

# Extract
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Extracted files to {extract_path}")


Extracted files to /content/images


In [12]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import Subset, DataLoader
from tqdm import tqdm

class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=2):
        super().__init__()
        # Feature extraction layers
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2,2),
        )

        # Classifier layers
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
num_classes = 2      # snake/spider
learning_rate = 0.001
batch_size = 64      # or 64 if memory allows
num_epochs = 10      # increase if needed

In [ ]:
!pip install opencv-python

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])

In [ ]:
dataset_path = 'images/'  # contains 'snake' and 'spider' subfolders
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
print(f"Classes: {full_dataset.classes}")  # ['snake', 'spider']

In [ ]:
from torch.utils.data import random_split, DataLoader

total_size = len(full_dataset)  # 5300 images
train_size = int(0.8 * total_size)  # 80% for training
test_size = total_size - train_size  # 20% for testing

train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Train samples: {len(train_dataset)}, Test samples: {len(test_dataset)}")

In [ ]:
model = CNN(in_channels=3, num_classes=num_classes).to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    print(f"Epoch [{epoch + 1}/{num_epochs}]")

    for batch_index, (data, targets) in enumerate(tqdm(train_loader)):
        data, targets = data.to(device), targets.to(device)

        # Forward
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Training accuracy
        _, preds = scores.max(1)
        correct += (preds == targets).sum().item()
        total += targets.size(0)

    train_acc = 100 * correct / total
    avg_loss = running_loss / len(train_loader)

    print(f"Epoch [{epoch + 1}/{num_epochs}] Loss: {avg_loss:.4f} | Train Accuracy: {train_acc:.2f}%")


In [ ]:
def check_accuracy(loader, model, loader_name="Data"):
    print(f"Checking accuracy on {loader_name}")

    num_correct = 0
    num_samples = 0
    model.eval()  # evaluation mode

    with torch.no_grad():  # no gradient needed
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum().item()  # convert to number
            num_samples += predictions.size(0)

    accuracy = 100 * num_correct / num_samples
    print(f"Got {num_correct}/{num_samples} correct -> Accuracy: {accuracy:.2f}%")

    model.train()  # back to training mode

# Usage
check_accuracy(train_loader, model, loader_name="Training Data")
check_accuracy(test_loader, model, loader_name="Test Data")